In [56]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import geopy
from geopy.geocoders import Nominatim
import calendar
import seaborn as sns
# Load data with longitude and latitude information into a pandas DataFrame

data = pd.read_csv('dummy_table.csv')
#location_counts = data.groupby(['lat','lon']).size().reset_index(name = 'count')

data['Date'] = pd.to_datetime(data['Date']).dt.date





In [57]:
crime_types = data.columns[17:36]  # Assuming the first column contains dates
#selected_crime_type = st.selectbox('Select a crime type', crime_types)
selected_crime_type = 'Missing Person'
filtered_df = data[['Date', 'lat','lon',selected_crime_type]]
#data
filtered_df
#crime_types

,Date,lat,lon,Missing Person
0,2023-05-30,52.509947,13.290002,0.0
1,2023-03-29,52.519284,13.170665,0.0
2,2023-05-02,52.518651,13.513130,0.0
3,2022-11-27,52.554893,13.337533,0.0
4,2022-07-06,52.440442,13.388289,0.0
...,...,...,...,...
224,2022-10-08,48.921296,7.481227,0.0
225,2023-02-05,52.544118,13.364232,0.0
226,2022-10-24,52.429205,13.229974,0.0
227,2022-09-14,52.495850,13.504417,0.0


In [58]:
location_counts = filtered_df.groupby(['lat', 'lon']).size().reset_index(name='Incident Count')
location_counts

,lat,lon,Incident Count
0,39.810415,-90.406061,1
1,48.921296,7.481227,3
2,52.375665,13.648855,1
3,52.393413,13.544935,1
4,52.395547,13.634656,1
...,...,...,...
135,52.627907,13.507135,2
136,52.628482,13.497434,1
137,53.092556,13.845087,1
138,53.594566,11.449994,1


In [40]:
#Group the incident that happens in the same day
grouped_df = data.groupby('Date')[selected_crime_type].sum().reset_index()
grouped_df


ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [36]:
pivoted_df = grouped_df.pivot_table(index='Date', values=selected_crime_type)
pivoted_df

,Missing Person
Date,
2022-05-30,0.0
2022-05-31,0.0
2022-06-02,0.0
2022-06-04,0.0
2022-06-08,0.0
...,...
2023-05-21,0.0
2023-05-23,0.0
2023-05-25,0.0


In [37]:
pivoted_df[f'{selected_crime_type}']

Date
2022-05-30    0.0
2022-05-31    0.0
2022-06-02    0.0
2022-06-04    0.0
2022-06-08    0.0
             ... 
2023-05-21    0.0
2023-05-23    0.0
2023-05-25    0.0
2023-05-28    0.0
2023-05-30    0.0
Name: Missing Person, Length: 203, dtype: float64

In [ ]:

# Create a density heatmap using Plotly Express
fig = px.density_mapbox(data, lat=data['lat'], lon=data['lon'], radius=pivoted_df[f'{selected_crime_type}'],  opacity=0.7, color_continuous_scale='YlOrRd',
                        zoom=10, mapbox_style='carto-positron',animation_frame=data['Date'])
address = st.text_input('Enter an address', '')
geolocator = Nominatim(user_agent='my_geocoder')
location = geolocator.geocode(address)

if location:
entered_lat = location.latitude
    entered_lon = location.longitude
    #Update the map with the new location
    fig.update_layout(mapbox_center={"lat": entered_lat, "lon": entered_lon}, mapbox_zoom=12)

    # Add a marker for the entered address
    fig.add_trace(go.Scattermapbox(
        lat=[entered_lat],
        lon=[entered_lon],
        mode='markers',
        marker={'size': 10, 'color': 'blue'}
    ))

    st.plotly_chart(fig)

selected_date = st.date_input('Select a date')



st.write('Heatmap of crime incidents')




#sns.heatmap(pivoted_df, cmap='YlOrRd', linewidths=0.5, annot=True, fmt='d')

# Display the Streamlit app
#st.pyplot()
# Display the heatmap in Streamlit
st.write('Entered address:', address)

# Display the selected date
st.write('Selected date:', selected_date)
